In [13]:
import os
import numpy as np
import pandas as pd
import gzip, h5py
import pickle
import glob

In [1]:
example_filename = '/n/holystore01/LABS/iaifi_lab/Lab/hbinney/ssm_files/Project8Sims_noNoise_dataMode_100k/Project8Sims_Row38_I_Q_noNoise_dataMode_5_6.pkl.gz'

In [3]:
with gzip.open(example_filename, 'rb') as f:
    data = pickle.load(f)

data.head()

,energy_eV,pitch_angle_deg,start_carrier_frequency_Hz,avg_carrier_frequency_Hz,avg_axial_frequency_Hz,slope_Hz,radius_m,radius_phase,output_ts_I,output_ts_Q
0,18596.234039,89.306699,2.590332e+10,2.590332e+10,4.439479e+06,3.571128e+08,0.002376,2.751532,"[7.843137254901975e-05, 0.002274509803921569, ...","[7.843137254901975e-05, 0.0013333333333333322,..."
1,18606.205344,89.192219,2.590273e+10,2.590273e+10,3.460143e+06,3.450138e+08,0.000437,2.386679,"[7.843137254901975e-05, 0.0018039215686274507,...","[7.843137254901975e-05, 0.0011764705882352927,..."
2,18692.952468,89.616905,2.590112e+10,2.590112e+10,1.835019e+06,3.854467e+08,0.001094,2.793499,"[7.843137254901975e-05, 0.002588235294117648, ...","[7.843137254901975e-05, 0.0011764705882352927,..."
3,18553.931071,89.738824,2.590375e+10,2.590375e+10,2.808667e+06,4.868447e+08,0.002779,2.533106,"[7.843137254901975e-05, 0.0057254901960784275,...","[7.843137254901975e-05, 0.004, -0.004, -0.0112..."
4,18513.081539,89.348122,2.590686e+10,2.590686e+10,4.626022e+06,4.530118e+08,0.004179,1.322241,"[7.843137254901975e-05, 0.005098039215686273, ...","[7.843137254901975e-05, 0.0038431372549019606,..."


In [4]:
data.to_parquet('test.parquet')

In [5]:
# To load:
df = pd.read_parquet('test.parquet')

In [31]:
file_list = glob.glob('/n/holystore01/LABS/iaifi_lab/Lab/hbinney/ssm_files/Project8Sims_noNoise_dataMode_100k/*.pkl.gz')
hdf5_path = '/n/holystore01/LABS/iaifi_lab/Lab/kyoon/project8/combined_data.hdf5'

if os.path.exists(hdf5_path):
    os.remove(hdf5_path)

# --- First pass: get columns, shapes, and total rows ---
with gzip.open(file_list[0], 'rb') as f:
    df = pickle.load(f)
columns = df.columns
shapes = {}
for col in columns:
    arr = df[col].to_numpy()
    if isinstance(arr[0], (list, tuple, np.ndarray)):
        arr0 = np.stack(arr)
        shapes[col] = arr0.shape[1:]
    else:
        shapes[col] = ()

total_rows = 0
for fname in file_list:
    with gzip.open(fname, 'rb') as f:
        df = pickle.load(f)
    total_rows += len(df)

# --- Second pass: create datasets and fill them ---
with h5py.File(hdf5_path, 'w') as h5f:
    dsets = {}
    for col in columns:
        shape = (total_rows,) + shapes[col]
        dsets[col] = h5f.create_dataset(col, shape=shape, dtype=np.float32)
    row_idx = 0
    for fname in file_list:
        with gzip.open(fname, 'rb') as f:
            df = pickle.load(f)
        rows = len(df)
        for col in columns:
            arr = df[col].to_numpy()
            if isinstance(arr[0], (list, tuple, np.ndarray)):
                arr = np.stack(arr).astype(np.float32)
            else:
                arr = arr.astype(np.float32)
            dsets[col][row_idx:row_idx+rows] = arr
        row_idx += rows

print(f'Saved all rows for each column as a single dataset in {hdf5_path}')

Saved all rows for each column as a single dataset in /n/holystore01/LABS/iaifi_lab/Lab/kyoon/project8/combined_data.hdf5


In [36]:
with h5py.File(hdf5_path, 'r') as f:
    print(f.keys())
    print(f['avg_axial_frequency_Hz'][0])
    arr = f['output_ts_I'][0]
    print(type(arr))

<KeysViewHDF5 ['avg_axial_frequency_Hz', 'avg_carrier_frequency_Hz', 'energy_eV', 'output_ts_I', 'output_ts_Q', 'pitch_angle_deg', 'radius_m', 'radius_phase', 'slope_Hz', 'start_carrier_frequency_Hz']>
4751968.0
<class 'numpy.ndarray'>
